In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

from acquire import get_telco_data

In [2]:
df = get_telco_data()

In [3]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
0,0003-MKNFE,Male,0,No,No,9,59.9,542.4,Yes,Yes,...,No,No,No,No,Yes,No,Month-to-month,Mailed check,DSL,No
1,0004-TLHLJ,Male,0,No,No,4,73.9,280.85,Yes,No,...,No,Yes,No,No,No,Yes,Month-to-month,Electronic check,Fiber optic,Yes
2,0011-IGKFF,Male,1,Yes,No,13,98.0,1237.85,Yes,No,...,Yes,Yes,No,Yes,Yes,Yes,Month-to-month,Electronic check,Fiber optic,Yes
3,0013-EXCHZ,Female,1,Yes,No,3,83.9,267.4,Yes,No,...,No,No,Yes,Yes,No,Yes,Month-to-month,Mailed check,Fiber optic,Yes
4,0013-MHZWF,Female,0,No,Yes,9,69.4,571.45,Yes,No,...,No,No,Yes,Yes,Yes,Yes,Month-to-month,Credit card (automatic),DSL,No


In [6]:
# step 1 split
train, test = train_test_split(df)

In [7]:
encoder = LabelEncoder()

encode_list = [
    'gender', 'partner', 'dependents', 'phone_service'
    , 'multiple_lines', 'online_security', 'online_backup'
    , 'device_protection', 'tech_support'
    , 'streaming_movies', 'streaming_tv', 'paperless_billing', 'churn'
    ]
               
               
               
for c in encode_list:
    train[c] = encoder.fit_transform(train[c])
    test[c] = encoder.transform(test[c])

In [8]:
train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
1075,1179-INLAT,1,0,0,0,3,19.45,69.25,1,0,...,1,1,1,1,1,1,One year,Mailed check,None,0
4221,4980-URKXC,1,0,1,0,36,20.85,720.05,1,0,...,1,1,1,1,1,0,One year,Bank transfer (automatic),None,0
6101,8894-JVDCV,0,0,0,0,62,115.05,7133.45,1,2,...,2,2,2,2,2,1,One year,Credit card (automatic),Fiber optic,0
2823,4086-ATNFV,0,0,1,1,34,60.80,2042.05,1,0,...,2,2,0,0,0,1,One year,Mailed check,DSL,0
6227,7317-GGVPB,1,0,1,0,71,108.60,7690.9,1,2,...,2,2,2,2,2,1,Two year,Credit card (automatic),Fiber optic,1


In [ ]:
df.info()

## Encode everything

In [ ]:
train, test = train_test_split(df)

In [ ]:
X_train = train.drop(columns=['customer_id', 'churn'])
y_train = train[['churn']]
X_test = test.drop(columns=['customer_id', 'churn'])
y_test = test[['churn']]

In [ ]:
f_selector = SelectKBest(chi2, k=2).fit(X_train, y_train)